### Imports

In [1]:
import os
import numpy as np
import pandas as pd

### Inputs

In [2]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_confidence = fusion_output_df['confidence'].values

In [3]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Kalman Filter

In [4]:
class KalmanFilter:
    def __init__(self, process_variance=1e-3, measurement_variance=1e-2, initial_estimate=0.5):
        self.x = initial_estimate       
        self.P = 1.0                    
        self.Q = process_variance       
        self.R = measurement_variance   

    def update(self, measurement):
        self.P += self.Q
        K = self.P / (self.P + self.R)
        self.x += K * (measurement - self.x)
        self.P *= (1 - K)
        return self.x

In [5]:
def apply_kalman(confidences):
    kf = KalmanFilter()
    smoothed = []
    for z in confidences:
        smoothed.append(kf.update(z))
    return np.array(smoothed)

### Apply

In [6]:
conf = apply_kalman(fusion_output_confidence)

In [7]:
tau = 0.5
cls = (conf >= tau).astype(int)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

In [8]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[cls[i]]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-87    3.8   38     0.58   1        Bom      0.95      
-79    6.7   30     0.63   1        Bom      0.96      
-60    3.1   37     0.39   1        Bom      0.89      
-54    1.2   38     0.26   1        Bom      0.91      
-47    2.7   26     0.16   1        Bom      0.92      
-88    6.8   68     0.97   1        Bom      0.94      
-61    2.9   25     0.01   1        Bom      0.94      
-41    3.9   16     0.09   1        Bom      0.95      
-62    1.3   25     0.27   1        Bom      0.94      
-54    2.2   2      0.09   1        Bom      0.95      
-83    7.1   53     0.71   1        Bom      0.95      
-42    3.6   18     0.22   1        Bom      0.95      
-85    5.1   81     0.96   1        Bom      0.94      
-67    8.3   62     0.45   1        Bom      0.94      
-60    3.6   33     0.79   1        Bom      0.94      
-53    1.3   2      0.39   1        Bom      0.9

### Save

In [9]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
